In [ ]:
import re
import nltk.corpus
import nltk.stem
import string

from nltk.corpus import words as nltk_words
from ipynb.fs.full.langdet import score_checker
from nltk.tokenize import word_tokenize

def remove_contractions(phrase):

     # remove contractions such as "they've" to "they have"

     phrase = re.sub(r"won\'t", "will not", phrase)
     phrase = re.sub(r"can\'t", "can not", phrase)
     phrase = re.sub(r"n\'t", " not", phrase)
     phrase = re.sub(r"\'re", " are", phrase)
     phrase = re.sub(r"\'s", " is", phrase)
     phrase = re.sub(r"\'d", " would", phrase)
     phrase = re.sub(r"\'ll", " will", phrase)
     phrase = re.sub(r"\'t", " not", phrase)
     phrase = re.sub(r"\'ve", " have", phrase)
     phrase = re.sub(r"\'m", " am", phrase)

     return phrase

def remove_content(text):

     # remove non-alphabet characthers, white spaces and links

     text = text.lower()
     text = text.encode('ascii', 'ignore').decode()
     text = re.sub("@\S+", " ", text)
     text = re.sub("https*\S+", " ", text)
     text = re.sub("#\S+", " ", text)
     text = remove_contractions(text)
     text = re.sub("\'\w+", '', text)
     text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
     text = re.sub(r'\w*\d+\w*', '', text)
     text = re.sub('\s{2,}', " ", text)
     
     return text

def is_english_word(word):

     # creates a dictionary using the nltk words corpus
    dictionary = dict.fromkeys(nltk_words.words(), None)

     # checks to see if the word parameter is apart of this dictionary and then returns a value
    try:
        x = dictionary[word]
        return True
    except KeyError:
        return False

def confirm_cs(text):

     # bool values to state what languages are present in a tweet
     is_urdu = False
     is_eng = False

     # tokenize the text
     tokenized_text = word_tokenize(text, language="english") 

     for item in tokenized_text:

          # confirm wether the word is urdu or english based of the language model and the english dictionary
          if((score_checker(item) > 12.389025170796918) and (is_english_word(item))):
               is_eng = True
          elif ((score_checker(item) < 12.389025170796918) and (not(is_english_word(item)))):
               is_urdu = True
          
          if (is_eng and is_urdu):
               break
     
     #if both are true, there is code-switching occuring
     if (is_eng and is_urdu):
          return True
     else:
          return False

In [ ]:
import re
import snscrape.modules.twitter as sntwitter
import pandas as pd

twitter_pakistan_collection=[]

# scrapes twitter data and narrows the region to Pakistan
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('near:"pakistan"').get_items()):
    
    # will stop once 500 tweets are scraped
    if len(twitter_pakistan_collection)> 500:
        break

    # confirms that the tweets are written in the roman script
    if (tweet.lang =='en'):

        #cleans the tweet before it is appended to data list
        tweet.content = remove_content(tweet.content)

        loc = 'pakistan'
        if (loc in (tweet.user.location).lower()):
            twitter_pakistan_collection.append([tweet.id, tweet.content]) 
            

In [ ]:
#creates a pandas data frame to display the tweets prior to filtering
tweets_Pakistan = pd.DataFrame(twitter_pakistan_collection, columns=['Tweet Id', 'Tweet'])

In [ ]:
#stores the tweets in a csv file so that they can be used later for further filtering
tweets_Pakistan.to_csv("TwitterData.csv")

In [ ]:
import csv

with open('TwitterData.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

del data[0]

# removes ID and tweet number
for item in data:
    del item[0:2]

In [ ]:
codemixed_data = []

for item in data:

    #iterates through the tweets and confirms wether or not there is a code-mixing in a tweet
    if (confirm_cs(item[0])):
        codemixed_data.append(item)
    

In [ ]:
codemixed_data

In [ ]:
def split_text(text):

     words = re.sub(r'[^\w\s]', '', text).split()
     
     return [word for word in words ]

words = split_text(str(codemixed_data))

# displays bigrams for the code-mixed rich tweets
ngram_analysis = pd.Series(nltk.ngrams(words, 2)).value_counts()